In [30]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from scipy.stats import randint as sp_randint
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from Lacoste_temporal_utils import add_monthly_UAE_holiday_count,get_lag_features, create_peak_calendar,merge_peak_calendar_info,create_monthly_seasonal_features,get_rate_of_sale_monthly,get_monthly_seasonality_index,get_moving_stats_features
import category_encoders as ce
import seaborn as sns

In [31]:
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
from typing import Optional, List, Tuple,  Callable, Dict, Union
import re
import os
import joblib
import holidays
import pickle
import fsspec

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# !pip install category_encoders

In [32]:
country = 'UAE'
gcs_path = "gs://trd-sf-ntb"
target="units"
brand = 'LACOSTE'
fc_horizon=19
experiment_name = "units" # or something like f"exp_{datetime.now():%Y%m%d_%H%M%S}"
file_input = "feature_store.parquet"


data_start_date = '2023-01-01'
train_end_date="2025-03-31"
cut_off_date='2025-07-01'
data_end_date = '2026-12-31'


# Starting date
dt = datetime.strptime(train_end_date, "%Y-%m-%d")
date=dt + relativedelta(day=1)+relativedelta(months=fc_horizon)
horizon_end_date=date.strftime("%Y-%m-%d")
fc_start_date=dt + relativedelta(day=1)+relativedelta(months=1)
fc_start_date=fc_start_date.strftime("%Y-%m-%d")

base_vars = {"key": "key",
             "target": "target",
             "date": "date"}

local_model_dir="saved_models/tuned/march"
os.makedirs(local_model_dir, exist_ok=True)

local_res_dir="saved_results/tuned/march"
os.makedirs(local_res_dir, exist_ok=True)

### Train-test split

In [33]:
full_path = f"{gcs_path}/{target}/{brand}/{target}/{file_input}"

df_ml=pd.read_parquet(full_path)

In [34]:
df_train=df_ml[pd.to_datetime(df_ml.date)<=train_end_date]
df_test=df_ml[(pd.to_datetime(df_ml.date)>train_end_date) &((pd.to_datetime(df_ml.date)<=horizon_end_date))]
# df_train=df_train[~df_train["festive_peak_flag"].isnull()]
x_train=df_train.drop(["target","key","date"],axis=1)
y_train=df_train["target"]
x_test=df_test.drop(["target","key","date"],axis=1)
y_test=df_test["target"]

In [35]:
x_train.isnull().sum()

holiday_count                   0
fourier_year_sin                0
fourier_year_cos                0
Lag12_y                         0
MA6_y                           0
STD6_y                          0
Seasonality_Index               0
UAE_shopping_peak_ratio         0
festive_peak_flag               0
ramadan_period                  0
public_holidays                 0
is_high_sales_intent_flag       0
shopping_event_code             0
fashion_week_season             0
deriv_1_pct                     0
deriv_2_pct                     0
deriv_3_pct                     0
deriv_6_pct                     0
monthly_max_qty                 0
monthly_min_qty                 0
deriv_1_flag                    0
total_cost_usd_facebook         0
total_cost_usd_google           0
total_cost_usd_instagram        0
total_cost_usd_tiktok           0
city                            0
is_ecom_fullfillment_loc        0
selling_square_ft               0
selling_square_ft_cat           0
distinct_disco

In [36]:
# df_test.reset_index(inplace=True)
df_test["Pred_Flag"]=1
df_train["Pred_Flag"]=0
# test_end_date=df_test.date.max()


In [37]:
df_input=pd.concat([df_train,df_test],axis=0)
main_df =df_train.copy()
future_rows_to_predict =df_test.copy()

In [38]:
df_input.isnull().sum()

key                               0
date                              0
target                          208
holiday_count                     0
fourier_year_sin                  0
fourier_year_cos                  0
Lag12_y                         208
MA6_y                           208
STD6_y                          208
Seasonality_Index               208
UAE_shopping_peak_ratio           0
festive_peak_flag                 0
ramadan_period                  208
public_holidays                 208
is_high_sales_intent_flag       208
shopping_event_code             208
fashion_week_season             208
deriv_1_pct                     208
deriv_2_pct                     208
deriv_3_pct                     208
deriv_6_pct                     208
monthly_max_qty                 208
monthly_min_qty                 208
deriv_1_flag                    208
total_cost_usd_facebook           0
total_cost_usd_google             0
total_cost_usd_instagram          0
total_cost_usd_tiktok       

In [39]:
future_rows_to_predict.columns[future_rows_to_predict.isnull().sum()>0]

Index(['target', 'Lag12_y', 'MA6_y', 'STD6_y', 'Seasonality_Index',
       'ramadan_period', 'public_holidays', 'is_high_sales_intent_flag',
       'shopping_event_code', 'fashion_week_season', 'deriv_1_pct',
       'deriv_2_pct', 'deriv_3_pct', 'deriv_6_pct', 'monthly_max_qty',
       'monthly_min_qty', 'deriv_1_flag', 'shopping_event_code_encode',
       'fashion_week_season_encode', 'uae_seasons_encode',
       'deriv_2_trend_flag_encode'],
      dtype='object')

In [40]:
x_train.columns

Index(['holiday_count', 'fourier_year_sin', 'fourier_year_cos', 'Lag12_y',
       'MA6_y', 'STD6_y', 'Seasonality_Index', 'UAE_shopping_peak_ratio',
       'festive_peak_flag', 'ramadan_period', 'public_holidays',
       'is_high_sales_intent_flag', 'shopping_event_code',
       'fashion_week_season', 'deriv_1_pct', 'deriv_2_pct', 'deriv_3_pct',
       'deriv_6_pct', 'monthly_max_qty', 'monthly_min_qty', 'deriv_1_flag',
       'total_cost_usd_facebook', 'total_cost_usd_google',
       'total_cost_usd_instagram', 'total_cost_usd_tiktok', 'city',
       'is_ecom_fullfillment_loc', 'selling_square_ft',
       'selling_square_ft_cat', 'distinct_discount_levels', 'avg_discount',
       'max_discount', 'min_discount', 'avg_promo_duration',
       'promo_days_in_month', 'percentage_products_on_promo',
       'shopping_event_code_encode', 'fashion_week_season_encode',
       'uae_seasons_encode', 'deriv_2_trend_flag_encode', 'channel_encode',
       'store_format_encode', 'key_encode', 'quarte

### Models - function

In [41]:
def random_forest(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_test: pd.DataFrame,
    model_parameters: dict,
    tuning: bool=True ,
    key:str = 'Test',
    inference_flag:bool = False,
    verbose: bool = True
):
    """
    random_forest model - multivariate model with two options .True for hyperparameter tuning and False for fixed values for parameters.
    Input is a dataframe with target column (independent variable) and features created from create_features() function.
    Args
        :param x_train: Training dataset with the regressors
        :param y_train: Target data
        :param x_test: Testing dataset with the regressors
        :param model_parameters: dictionary of parameters
        :param tuning: hyperparameter setting
    Returns:
        list containing the forecast values
    """
    if tuning==True:
        params = model_parameters["True"]
        reg = parameter_tuning(
            x_train = x_train,
            y_train = y_train,
            regressor = RandomForestRegressor(),
            params = params,
            n_iter = 10
        )
        model = reg.best_estimator_
        model.fit(x_train,y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)
        return model
    else:
        params = model_parameters["False"]
        n_estimators = params["n_estimators"]
        max_depth = params["max_depth"]
        model = RandomForestRegressor(
            n_estimators = n_estimators,
            max_depth = max_depth,
        )
        model.fit(x_train,y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)
        return model

def xgboost(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_test: pd.DataFrame,
    model_parameters: dict,
    tuning: bool = True,
    key:str = 'Test',
    inference_flag:bool = False,
    verbose: bool = True
):
    """
    xg_boost model - multivariate model with two options .True for hyperparameter tuning and False for fixed values for parameters.
    """
    # Replace infinite values with NaN
    x_train = x_train.replace([np.inf, -np.inf], np.nan)
    x_test = x_test.replace([np.inf, -np.inf], np.nan)
    
    if tuning == True:
        params = model_parameters["True"].copy()
        params['missing'] = np.nan  # Ensure missing is set to handle NaN
        
        reg = parameter_tuning(
            x_train = x_train,
            y_train = y_train,
            regressor = XGBRegressor(enable_categorical=True, missing=np.nan),
            params = params,
            n_iter = 10
        )
        model = reg.best_estimator_
        model.fit(x_train, y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)
    else:
        params = model_parameters["False"].copy()
        params['missing'] = np.nan  # Ensure missing is set to handle NaN
        
        model = XGBRegressor(
            enable_categorical=True,
            missing=np.nan,
            n_estimators = params["n_estimators"],
            learning_rate = params["learning_rate"],
            max_depth = params["max_depth"],
            n_jobs = params["n_jobs"],
            alpha = params["alpha"],
            tree_method = params.get("tree_method", 'hist')
        )
        model.fit(x_train, y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)

    return model

def lgbm(
    x_train: pd.DataFrame,
    y_train: pd.DataFrame,
    x_test: pd.DataFrame,
    model_parameters: dict,
    tuning: bool = True,
    key:str = 'Test',
    inference_flag:bool = False,
    verbose: bool = True
):
    """
    LightGBM forecast model
    """
    # Replace infinite values with NaN
    x_train = x_train.replace([np.inf, -np.inf], np.nan)
    x_test = x_test.replace([np.inf, -np.inf], np.nan)
    
    if tuning == True:
        params = model_parameters["True"].copy()
        params.pop('verbose', None)
        reg = parameter_tuning(
            x_train = x_train,
            y_train = y_train,
            regressor = LGBMRegressor(),
            params = params,
            n_iter = 10
        )
        model = reg.best_estimator_
        model.fit(x_train,y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)
    else:
        params = model_parameters["False"]
        model = LGBMRegressor(
            objective = params["objective"],
            boosting_type = params["boosting_type"],
            num_leaves= params["num_leaves"],
            learning_rate = params["learning_rate"],
            force_col_wise = params["force_col_wise"],
            verbose= params["verbose"],
            seed= params["seed"]
        )
        model.fit(x_train,y_train)
        if inference_flag == True:
            return model
        # pred = model.predict(x_test)
    return model

In [42]:
multivariate_model_list = ['xgboost', 'lgbm', 'random_forest']

model_test = {
    'xgboost': xgboost,
    'lgbm': lgbm,
    'random_forest': random_forest
}

model_parameters_dict = {
    "lgbm": {
        "True": {
            'objective': ['regression'],
            'metric': ['mse'],
            'boosting_type': ['gbdt'],
            'num_leaves': [25,30,35,40],
            'learning_rate': [0.01,0.02,0.05,0.07],
            'feature_fraction': [0.5,0.7,0.9],
            'bagging_fraction': [0.7,0.75,0.8],
            'bagging_freq': [2,5,7],
            'verbose': 0,
            'seed': [42]
        },
        "False": {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 4,
            'learning_rate': 0.05,
            'force_col_wise': True,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0,
            'seed': 42
        }
    },
    "xgboost": {
        "True": {
            "max_depth": list(range(3, 10 + 1)),
            "learning_rate": [0.3, 0.2, 0.1, 0.05],
            "tree_method": ['hist'],
            "missing": [np.nan],  # Explicitly handle missing values
            "n_estimators": list(range(100, 300 + 1, 20)),
            "gamma": [0, 0.1, 0.01, 0.001],
            "subsample": [0.9, 0.8, 0.7],
            "colsample_bytree": [0.9, 0.8, 0.7],
            "colsample_bylevel": [0.9, 0.8, 0.7],
            "scale_pos_weight": [0.9, 0.8, 0.7],
            "importance_type": ['gain', 'weight', 'cover', 'total_gain', 'total_cover']
        },
        "False": {
            "n_estimators": 500,
            "learning_rate": 0.05,
            "max_depth": 6,
            "n_jobs": 1,
            "alpha": 0.1,
            "tree_method": 'hist',
            "missing": np.nan  # Explicitly handle missing values
        }
    },
    "random_forest": {
        "True": {
            "n_estimators": list(range(100, 300 + 1, 10)),
            "max_depth": list(range(3, 15 + 1)),
            "min_samples_split": [2, 3, 4, 5, 6],
            "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7],
            "max_features": [None, 'sqrt', 'log2', 0.9, 0.85, 0.75],
            "bootstrap": [True, False]
        },
        "False": {
            "n_estimators": 50,
            "max_depth": 200,
        }
    }
}

In [43]:
def parameter_tuning(x_train, y_train, regressor, params, n_iter=10):
    """
    Perform RandomizedSearchCV for hyperparameter tuning with proper parameter handling
    """
    # Replace infinite values if not already done
    x_train = x_train.replace([np.inf, -np.inf], np.nan)
    
    # Make a copy of params to avoid modifying the original
    search_params = params.copy()
    
    # For XGBoost, ensure missing parameter is properly formatted
    if isinstance(regressor, XGBRegressor):
        if 'missing' in search_params:
            # If missing is specified as a single value, wrap it in a list
            if not isinstance(search_params['missing'], (list, np.ndarray)):
                search_params['missing'] = [search_params['missing']]
    
    search = RandomizedSearchCV(
        estimator=regressor,
        param_distributions=search_params,
        scoring='neg_root_mean_squared_error',
        n_iter=n_iter,
        cv=3,
        verbose=1,
        n_jobs=-1,
        random_state=42
    )
    search.fit(x_train, y_train)
    return search

### Train champion challenger model

In [44]:
def multivariate_model_sprint(df_input,x_train,y_train,x_test,y_test):
    final = pd.DataFrame()
    for model_name in multivariate_model_list:
        print(x_test.columns)
     
        model_func = model_test[model_name]
        print(model_func)
        # Initialize a DataFrame to hold the results for the current model
        master = pd.DataFrame(columns=[base_vars['date'], base_vars['key'], 'forecast'])

        # Call the current model function to generate forecasts
        model= model_func(
            x_train.drop([base_vars['key'], base_vars['date']], axis=1),
            y_train.drop([base_vars['key']], axis=1),
            x_test.drop([base_vars['key'], base_vars['date']], axis=1),
            model_parameters=model_parameters_dict[model_name],  # Model-specific parameters
            tuning=True  # Tuning flag for hyperparameters
        )
        expected_features = X_train_1.drop([base_vars['key'], base_vars['date']], axis=1).columns.tolist()
        model_package = (model, expected_features)
        # Record the status as success
        
    
        
        local_model_path = os.path.join(local_model_dir, f"{model_name}_model.pkl")
        with open(local_model_path, 'wb') as f:
            pickle.dump(model_package, f)

In [45]:
def multivariate_model_sprint(df_input,x_train,y_train,x_test,y_test):
    final = pd.DataFrame()
    for model_name in multivariate_model_list:
        print(x_test.columns)
     
        model_func = model_test[model_name]
        print(model_func)
        # Initialize a DataFrame to hold the results for the current model
        master = pd.DataFrame(columns=[base_vars['date'], base_vars['key'], 'forecast'])

        # Call the current model function to generate forecasts
        model= model_func(
            x_train.drop([base_vars['key'], base_vars['date']], axis=1),
            y_train.drop([base_vars['key']], axis=1),
            x_test.drop([base_vars['key'], base_vars['date']], axis=1),
            model_parameters=model_parameters_dict[model_name],  # Model-specific parameters
            tuning=True  # Tuning flag for hyperparameters
        )
        expected_features = X_train_1.drop([base_vars['key'], base_vars['date']], axis=1).columns.tolist()
        model_package = (model, expected_features)
        # Record the status as success
        
    
        
        local_model_path = os.path.join(local_model_dir, f"{model_name}_model.pkl")
        with open(local_model_path, 'wb') as f:
            pickle.dump(model_package, f)



In [46]:
feature_list=['Lag12_y', 'MA6_y', 'STD6_y','Seasonality_Index']

In [47]:
# for model_name in ["xgboost","lightgbm","random_forest"]:
df_train=df_ml[pd.to_datetime(df_ml.date)<=train_end_date]
df_test=df_ml[(pd.to_datetime(df_ml.date)>train_end_date) &((pd.to_datetime(df_ml.date)<=data_end_date))]

X_train_1 = df_train.drop([base_vars['target']],axis=1) # Training features
X_test_1 =df_test.drop([ base_vars['target']],axis=1)   # Test features
Y_train_1 =df_train[[base_vars['target'],base_vars["key"]]] # Training target
Y_test = df_test[[base_vars['target'],base_vars["key"]]]  # Test target

In [53]:
multivariate_model_sprint(df_input,X_train_1,Y_train_1,X_test_1,Y_test)

Index(['key', 'date', 'holiday_count', 'fourier_year_sin', 'fourier_year_cos',
       'Lag12_y', 'MA6_y', 'STD6_y', 'Seasonality_Index',
       'UAE_shopping_peak_ratio', 'festive_peak_flag', 'ramadan_period',
       'public_holidays', 'is_high_sales_intent_flag', 'shopping_event_code',
       'fashion_week_season', 'deriv_1_pct', 'deriv_2_pct', 'deriv_3_pct',
       'deriv_6_pct', 'monthly_max_qty', 'monthly_min_qty', 'deriv_1_flag',
       'total_cost_usd_facebook', 'total_cost_usd_google',
       'total_cost_usd_instagram', 'total_cost_usd_tiktok', 'city',
       'is_ecom_fullfillment_loc', 'selling_square_ft',
       'selling_square_ft_cat', 'distinct_discount_levels', 'avg_discount',
       'max_discount', 'min_discount', 'avg_promo_duration',
       'promo_days_in_month', 'percentage_products_on_promo',
       'shopping_event_code_encode', 'fashion_week_season_encode',
       'uae_seasons_encode', 'deriv_2_trend_flag_encode', 'channel_encode',
       'store_format_encode', 'key_e

In [ ]:
hello

### ensemble models - stacking approach

### Recursive features

In [54]:
def feature_combine_pd(dataframe: pd.DataFrame,
                       dfu_columns: list[str],
                       feature_list: dict[str, any],
                       key: str,
                       ds: str,
                       target: str) -> pd.DataFrame:

    temp_df = dataframe.copy()

    temp_df = get_lag_features(temp_df, [12], key=key, date_col=ds, target=target)
    temp_df = get_moving_stats_features(temp_df, months_back=[6])
    temp_df = get_monthly_seasonality_index(temp_df, 'date', 'target')
    # temp_df=create_monthly_seasonal_features(temp_df,'date', 'target','key')
    
    # Add year column if not already there
    if 'year' not in temp_df.columns:
        temp_df['year'] = pd.to_datetime(temp_df[ds]).dt.year

    # Generate & merge rate_of_sale
    rate_of_sale_df = get_rate_of_sale_monthly(temp_df, [key])
    temp_df = temp_df.merge(rate_of_sale_df, on=[key, 'year'], how='left')

    return temp_df


In [55]:
def forecast_loop_store(key,df_in,model_pred, expected_features,
                  feature_list,model_name):
    future_rows_to_predict=df_in[df_in["Pred_Flag"]==1].copy()
    future_rows_to_predict.sort_values("date",inplace=True)
    main_df=df_in[df_in["Pred_Flag"]==0].copy()
    main_df.sort_values("date",inplace=True)
    
    for i in range(len(future_rows_to_predict)):
        data_sv=[]
        row_to_forecast = future_rows_to_predict.iloc[[i],:].copy()
        date=row_to_forecast["date"].values[0]
        # print(date)
        pred_flag=row_to_forecast["Pred_Flag"].values[0]
        temp_df = pd.concat([main_df, row_to_forecast], ignore_index=True)
        # print(temp_df.columns)
        temp_df.drop(columns=feature_list, errors='ignore', inplace=True)

        # Generate features
        feature_df = feature_combine_pd(temp_df, ['key'], feature_list, 'key', 'date', 'target')
        model_input = feature_df.reindex(columns=expected_features, fill_value=0)

        row_pred = model_input.iloc[[-1]]  # Last row is the forecast row
        column_name=expected_features.copy()
        column_name.extend(["target",'key', 'date','Pred_Flag'])
        # Predict
        pred = model_pred.predict(row_pred)
        record=list(np.concatenate([row_pred.values.flatten(),pred.flatten()]).reshape(len(row_pred.values[0])+1))
        record.append(key)
        record.append(date)
        record.append(pred_flag)
        data_sv.append(record)
       
        # Update prediction in the future dataframe
        future_rows_to_predict.iloc[i, future_rows_to_predict.columns.get_loc('target')] = pred[0]
        # Append predicted row to df for next iteration
        row_to_forecast['target'] = pred[0]
        df_res=pd.DataFrame(data_sv,columns=column_name)
        main_df = pd.concat([main_df[column_name], df_res], ignore_index=True)
        
    # df_res=pd.DataFrame(data_sv,columns=column_name)
    # df_res2=pd.concat([main_df[column_name],df_res],ignore_index=True)

    local_key_forecast_path = os.path.join(local_res_dir, f"{key}_{model_name}_forecast_.csv")
    
    print(f"{local_key_forecast_path}")
    main_df.to_csv(local_key_forecast_path)
    
     # Prepare forecast_df with predictions
    forecast_df = future_rows_to_predict[['key', 'date', 'target']].copy()
    forecast_df.rename(columns={'target': 'forecast'}, inplace=True)
    forecast_df['run_status'] = f"{df_in['key'].iloc[0]} success"
    return forecast_df


# model inference

In [56]:
def multivariate_model_sprint_output(df_input,x_train,y_train,x_test,y_test):
    final = pd.DataFrame()
    all_keys=x_test["key"].unique().tolist()
    for model_name in multivariate_model_list:
        print(x_test.columns)
        model_func = model_test[model_name]
        print(model_func)
        # Initialize a DataFrame to hold the results for the current model
        master = pd.DataFrame(columns=[base_vars['date'], base_vars['key'], 'forecast'])
        local_model_path = os.path.join(local_model_dir, f"{model_name}_model.pkl")
        with open(local_model_path, 'rb') as f:
            model1 = pickle.load(f)
            print(model1)
            model=model1[0]
            expected_features=model1[1]
        # print(model)
        # generating forecast for stores
        dff=pd.DataFrame({})
        for k in all_keys:
            df_key=df_input[df_input["key"]==k].copy()
            # print(df_key.head())
            df_forecast=forecast_loop_store(k,df_key,model ,expected_features,feature_list,model_name)
            dff=pd.concat([dff,df_forecast],axis=0,ignore_index=True)


        merged_df=pd.merge(df_test[[base_vars['date'], base_vars['key'],base_vars['target']]],dff,how="left",on=[base_vars['date'], base_vars['key']])
        # apr_may_jun_df=merged_df[merged_df["date"]<"2025-07-01"].reset_index(drop=True)

        ovrll_forecast_path = os.path.join(local_res_dir, f"forecast_jun2026_{model_name}.csv")
        dff.to_csv(ovrll_forecast_path,index=False)
        dff['run_status_' + model_name] = 'success'

        dff=dff[master.columns]
        # Concatenate the current model results to the master DataFrame
        master = pd.concat([master, dff], axis=0)

        # Drop rows with NaN in the key column to clean up the results
        master.dropna(subset=[base_vars['key']], inplace=True)

        # Rename the forecast column to include the model name
        master.rename(columns={'forecast': f'forecast_{model_name}'}, inplace=True)

        # Ensure the date column is in datetime format
        master[base_vars['date']] = pd.to_datetime(master[base_vars['date']])
        # master.to_csv('inter_out.csv',index=False)

        # Merge the results into the final DataFrame
        if final.empty:
            final = master.copy()  # If final is empty, initialize it with master
        else:
            if base_vars['target'] in final.columns.tolist():
                final.drop('target',axis=1,inplace=True)
            final = pd.merge(final, master, on=[base_vars['key'], base_vars['date']], how='inner')  # Merge
        
        # Merge the predicted values with the actual data
        final = pd.merge(final, df_input[df_input["Pred_Flag"]==1][[base_vars['date'], base_vars['key'],base_vars['target']]], on=[base_vars['key'], base_vars['date']], how='left')

        
    return final

In [57]:
completed_fc=multivariate_model_sprint_output(df_input,X_train_1,Y_train_1,X_test_1,Y_test)

Index(['key', 'date', 'holiday_count', 'fourier_year_sin', 'fourier_year_cos',
       'Lag12_y', 'MA6_y', 'STD6_y', 'Seasonality_Index',
       'UAE_shopping_peak_ratio', 'festive_peak_flag', 'ramadan_period',
       'public_holidays', 'is_high_sales_intent_flag', 'shopping_event_code',
       'fashion_week_season', 'deriv_1_pct', 'deriv_2_pct', 'deriv_3_pct',
       'deriv_6_pct', 'monthly_max_qty', 'monthly_min_qty', 'deriv_1_flag',
       'total_cost_usd_facebook', 'total_cost_usd_google',
       'total_cost_usd_instagram', 'total_cost_usd_tiktok', 'city',
       'is_ecom_fullfillment_loc', 'selling_square_ft',
       'selling_square_ft_cat', 'distinct_discount_levels', 'avg_discount',
       'max_discount', 'min_discount', 'avg_promo_duration',
       'promo_days_in_month', 'percentage_products_on_promo',
       'shopping_event_code_encode', 'fashion_week_season_encode',
       'uae_seasons_encode', 'deriv_2_trend_flag_encode', 'channel_encode',
       'store_format_encode', 'key_e

In [58]:
completed_fc

,date,key,forecast_xgboost,forecast_lgbm,forecast_random_forest,target
0,2025-04-01,52003,2568.377686,2648.402668,2740.789787,2642.0
1,2025-05-01,52003,2454.080078,2798.185040,2689.648159,2723.0
2,2025-06-01,52003,3134.905273,3299.536865,3108.101343,2198.0
3,2025-07-01,52003,3440.908203,1816.886047,2467.371696,NaN
4,2025-08-01,52003,3170.845703,1529.307913,2046.808630,NaN
...,...,...,...,...,...,...
242,2026-06-01,52086,1860.349121,212.119674,546.340919,NaN
243,2026-07-01,52086,1995.850708,254.892491,643.678613,NaN
244,2026-08-01,52086,2104.596191,152.624459,576.889237,NaN
245,2026-09-01,52086,1841.534790,135.279798,537.651423,NaN


In [83]:
forecast_df=completed_fc[( ~completed_fc["target"].isnull()) & (completed_fc["key"]!='52004')]

In [84]:
forecast_df.drop_duplicates(subset=["date","key"],inplace=True)

In [85]:
forecast_df.isnull().sum()

date                      0
key                       0
forecast_xgboost          0
forecast_lgbm             0
forecast_random_forest    0
target                    0
dtype: int64

In [ ]:
# forecast_df=forecast_df[(forecast_df["date"]!="2025-07-01")]

# forecast_jul=forecast_df[(forecast_df["date"]=="2025-07-01")]

### Metrics

In [86]:
# Calculate WMAPE
def weighted_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true) * 100

def results(df_res,actuals:str,forecast:str,model_name:str,key:str,date:str):
    """ metrics - mae, rmse, r2, std_dev are obtained on predicted values
        Plot of target v/s forecasted is obtained on test data 
    """
    df_temp=df_res.copy()
    y_test=df_temp[actuals]
    y_pred=df_temp[forecast]
    print(f"---------{model_name}------------")
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    std_dev = y_test.std()
    rmse_std_ratio = rmse / std_dev

    wmape = weighted_mean_absolute_percentage_error(y_test, y_pred)

    # Output metrics
    print("\nModel Evaluation Metrics:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R² Score: {r2:.4f}")
    print(f"Standard Deviation of Test Data: {std_dev:.2f}")
    print(f"RMSE/Std Dev Ratio: {rmse_std_ratio:.2f}")
    print(f"WMAPE: {wmape:.2f}%")

    # # Plot feature importance
    # plt.figure(figsize=(10, 6))
    # xgb.plot_importance(xgb_model, max_num_features=20)
    # plt.title('XGBoost Feature Importance')
    # plt.tight_layout()
    # plt.show()
    index_plot=df_temp[key].astype(str)+"_"+\
    df_temp[date].astype(str).str.replace(r'-\d{2}$', '', regex=True)
    
    # Plot actual vs predicted
    plt.figure(figsize=(12, 6))
    plt.plot(range(len(y_test)), y_test, label='Actual', linewidth=2)
    plt.plot(range(len(y_test)) ,y_pred, label='Predicted', linestyle='--', linewidth=2)
    plt.title(f'Monthly Store Units:({model_name})')
    plt.xticks(range(len(y_test)),index_plot)
    plt.xlabel('Store_Month')
    plt.legend()
    plt.ylabel('Unit count')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()



In [87]:
forecast_df.date.min(),forecast_df.date.max()

(Timestamp('2025-04-01 00:00:00'), Timestamp('2025-06-01 00:00:00'))

In [88]:
for forecast_col in ['forecast_xgboost','forecast_lgbm','forecast_random_forest']:
    
    model_name=forecast_col.split("_")[1]
    forecast_df[f'{model_name}_row_mape'] = (
            np.abs(forecast_df[base_vars['target']] - forecast_df[forecast_col]) / forecast_df[base_vars['target']]
        ) * 100
    
forecast_df=forecast_df.rename(columns={"random_row_mape":"random_forest_row_mape"})

In [89]:
forecast_list=['forecast_xgboost','forecast_lgbm','forecast_random_forest']
 
def champion_challenger_res(df_input,forecast_list):
    df=df_input.copy()
    model_map={"xgboost":"forecast_xgboost","lgbm":"forecast_lgbm","random_forest":"forecast_random_forest"}
    df_cc=pd.DataFrame({})
    df["old_model_name"]=df.apply(lambda row: sorted([("xgboost",row["xgboost_row_mape"]),("lgbm",row["lgbm_row_mape"]),("random_forest",row["random_forest_row_mape"])], key=lambda x: x[1],reverse=False)[0][0],axis=1)
    df["old_best_forecast"]=df.apply(lambda row: sorted([(row["forecast_xgboost"],row["xgboost_row_mape"]),(row["forecast_lgbm"],row["lgbm_row_mape"]),(row["forecast_random_forest"],row["random_forest_row_mape"])], key=lambda x: x[1],reverse=False)[0][0],axis=1)
    print(df["old_model_name"].value_counts())
    df_cc=df["old_model_name"].value_counts()
    for col in forecast_list:
 
        df_cc[f"mape_{col}"]=df.apply(lambda x: x[model_map[x["old_model_name"]]],axis=1)
 
        # print(weighted_mean_absolute_percentage_error(df['target'],df['best_forecast']))
       # print(f"{col} wmape {weighted_mean_absolute_percentage_error(df['target'],df[col])}")
        mape_value = mean_absolute_percentage_error(df['target'],df[col])* 100
        print(f" {col} mape: {mape_value:.2f}%")
    # print(weighted_mean_absolute_percentage_error(df['target'],df['best_forecast']))
    print(f" old  store month mape {mean_absolute_percentage_error(df['target'],df['old_best_forecast'])* 100}")
    return df
forecast_df=champion_challenger_res(forecast_df,forecast_list)

old_model_name
xgboost          15
lgbm             12
random_forest     9
Name: count, dtype: int64
 forecast_xgboost mape: 19.97%
 forecast_lgbm mape: 21.56%
 forecast_random_forest mape: 23.22%
 old  store month mape 15.855759772010336


In [90]:
model_name =["xgboost","lgbm","random_forest"]
def grouping_df(df):
    df_input=df.copy()
    key_model_map={}
    grp_forecast=df_input.groupby("key").agg(forecast_xgboost=("forecast_xgboost","sum"),forecast_lgbm=("forecast_lgbm",'sum'),\
                                   forecast_random_forest=("forecast_random_forest",'sum'),old_best_forecast=('old_best_forecast','sum'),target=('target','sum'))
    for model in model_name:
        grp_forecast[f"{model}_row_mape"]=np.abs(grp_forecast["target"]-grp_forecast[f"forecast_{model}"])/grp_forecast["target"]*100
    grp_forecast["old_best_forecast_row_mape"]=np.abs(grp_forecast["target"]-grp_forecast["old_best_forecast"])/grp_forecast["target"]*100
    grp_forecast["new_model_name"]=grp_forecast.apply(lambda row: sorted([("xgboost",row["xgboost_row_mape"]),("lgbm",row["lgbm_row_mape"]),("random_forest",row["random_forest_row_mape"])], key=lambda x: x[1],reverse=False)[0][0],axis=1)
    grp_forecast.reset_index(inplace=True)
    for i,j in grp_forecast[["key","new_model_name"]].values :
        key_model_map[i]=j
    grp_forecast["new_best_forecast"]=grp_forecast.apply(lambda row: sorted([(row["forecast_xgboost"],row["xgboost_row_mape"]),(row["forecast_lgbm"],row["lgbm_row_mape"]),(row["forecast_random_forest"],row["random_forest_row_mape"])], key=lambda x: x[1],reverse=False)[0][0],axis=1)
    grp_forecast["new_best_forecast_row_mape"]=np.abs(grp_forecast["target"]-grp_forecast["new_best_forecast"])/grp_forecast["target"]*100
 
    
    print(f"old approach store mape {grp_forecast['old_best_forecast_row_mape'].mean()}")
    print(f"new approach store mape {grp_forecast['new_best_forecast_row_mape'].mean()}")
    return grp_forecast
key_df=grouping_df(forecast_df)

old approach store mape 10.85525645148194
new approach store mape 11.2489269564191


In [91]:
model_map={"xgboost":"forecast_xgboost","lgbm":"forecast_lgbm","random_forest":"forecast_random_forest"}

forecast_df=pd.merge(forecast_df,key_df[["key","new_model_name"]],how="inner",on="key")

forecast_df["new_best_forecast"]=forecast_df.apply(lambda x: x[model_map[x["new_model_name"]]],axis=1)

print(f" new approach store month mape {mean_absolute_percentage_error(forecast_df['target'],forecast_df['new_best_forecast'])* 100}")
 

 new approach store month mape 18.612822235039282


In [93]:
key_df

,key,forecast_xgboost,forecast_lgbm,forecast_random_forest,old_best_forecast,target,xgboost_row_mape,lgbm_row_mape,random_forest_row_mape,old_best_forecast_row_mape,new_model_name,new_best_forecast,new_best_forecast_row_mape
0,52003,8157.363037,8746.124573,8538.539290,8446.152170,7563.0,7.858826,15.643588,12.898840,11.677273,xgboost,8157.363037,7.858826
1,52009,5327.412109,5226.298337,5103.235183,5071.743791,4913.0,8.435011,6.376925,3.872078,3.231097,random_forest,5103.235183,3.872078
2,52010,3514.829895,3448.770347,3440.526409,3267.684747,2545.0,38.107265,35.511605,35.187678,28.396257,random_forest,3440.526409,35.187678
3,52012,3071.445007,3219.080036,3388.848516,3071.445007,2627.0,16.918348,22.538258,29.000705,16.918348,xgboost,3071.445007,16.918348
4,52020,1868.823303,1879.476535,1913.043975,1863.779823,1758.0,6.303942,6.909928,8.819339,6.017055,xgboost,1868.823303,6.303942
5,52043,5593.515381,5897.189142,5628.836916,5498.685929,5322.0,5.101755,10.807763,5.765444,3.319916,xgboost,5593.515381,5.101755
6,52052,11588.588135,12102.468838,11448.784499,11447.329904,9175.0,26.306138,31.907017,24.782392,24.766538,random_forest,11448.784499,24.782392
7,52071,12489.620605,12860.368120,12456.018214,12714.907915,13003.0,3.948161,1.096915,4.206581,2.215582,lgbm,12860.368120,1.096915
8,52074,2922.492065,2947.790329,2943.304001,2959.933834,2973.0,1.698888,0.847954,0.998856,0.439494,lgbm,2947.790329,0.847954
9,52082,1226.786224,1071.414787,1378.372793,1196.460744,1180.0,3.964934,9.202137,16.811254,1.394978,xgboost,1226.786224,3.964934
